In [ ]:
%matplotlib inline

import sys
import datacube
import numpy as np
import pandas as pd
import xarray as xr
import rasterio
import fiona
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.cm import RdYlGn
from datacube.utils import geometry
from matplotlib.patches import Patch
import matplotlib._color_data as mcd
from matplotlib.colors import ListedColormap
from datacube.utils.cog import write_cog
from skimage.filters import threshold_otsu


sys.path.append('../Scripts')
from odc.algo import xr_geomedian
from deafrica_bandindices import calculate_indices
from deafrica_dask import create_local_dask_cluster
from deafrica_datahandling import load_ard, wofs_fuser, mostcommon_crs
from deafrica_plotting import display_map, map_shapefile, rgb
from deafrica_spatialtools import xr_rasterize, xr_vectorize

In [ ]:
def calculate_area(resoltion):
    pixel_length = resoltion  # in metres
    m_per_km = 1000  # conversion from metres to kilometres
    area_per_pixel = pixel_length**2 / m_per_km**2
    return area_per_pixel

In [ ]:
def convert_file(vector_file):

    if vector_file.split(".")[-1] == 'kml':

        gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'

        # Read shapefile into a GeoDataFrame
        gdf = gpd.read_file(vector_file, driver='KML')

    else:
        gdf = gpd.read_file(vector_file)

    geom = geometry.Geometry(gdf.unary_union, gdf.crs)

    map_shapefile(gdf, attribute=gdf.columns[0], fillOpacity=0, weight=3)

    return gdf, geom

In [ ]:
def process_data(dc, geom, baseline_year, analysis_year):

    #Create a query object
    product = ['s2_l2a']

    query = {
        'group_by': 'solar_day',
        'geopolygon' : geom,    
    }
    
    # Identify the most common projection system in the input query
    output_crs = mostcommon_crs(dc=dc, product=product, query=query)

    ds = load_ard(dc=dc,
                  products=product, 
                  output_crs=output_crs,
                  dask_chunks={'time':1,'x':2000,'y':2000},
                  measurements=["red","green","blue","nir"],
                  time=(f'{baseline_year}', f'{analysis_year}'),
                  resolution = (-10, 10),
                  scaling='normalised',
                  **query)
    
    ds_wofs_baseline = dc.load(product=["ga_ls8c_wofs_2_annual_summary"],
                               time=(f'{baseline_year}', f'{analysis_year}'),
                               output_crs=output_crs,
                               resolution = (-10, 10),
                               **query
                              )

    ds_geomedian = ds.groupby('time.year').map(xr_geomedian)

    ds_geomedian = calculate_indices(ds_geomedian, ['NDVI'], collection='c1')

    ds_geomedian = ds_geomedian.compute()

    # Convert the polygon to a raster that matches our imagery data

    mask = xr_rasterize(gdf, ds_geomedian)

        # Mask dataset to set pixels outside the polygon to `NaN`
    ds_geomedian = ds_geomedian.where(mask)
    
    mid = int(round(ds_geomedian.year.size / 2, 0))
    
    rgb(ds_geomedian, index_dim='year', index=[0, mid, -1])
    
   
    
    count_wet=ds_wofs_baseline.count_wet.where(ds_wofs_baseline.count_wet>=0)
    count_clear=ds_wofs_baseline.count_clear.where(ds_wofs_baseline.count_clear>=0)
    water_frequency=count_wet/count_clear
    water_frequency = water_frequency.compute()
    water_frequency_boolean = xr.where(water_frequency > 0.1, True, False)
    
    water_frequency_sum = water_frequency_boolean.sum('time')
    water_frequency_sum = water_frequency_sum.where(mask)
            
    return ds_geomedian, water_frequency_sum

In [ ]:
def run_vegetation_loss(ds_geomedian):
    
    ds_shift = ds_geomedian.NDVI.shift(year=1)
    
    ds_change_ndvi = ds_geomedian.NDVI - ds_shift
    
    threshold = threshold_otsu(ds_change_ndvi.fillna(0).values)
    
    vegetation_loss = ds_change_ndvi < threshold
    vegetation_loss = vegetation_loss.where(vegetation_loss == True)
    
    
    count_vegetation_loss = vegetation_loss.count(dim=['x', 'y'])

    vegetation_loss_area = count_vegetation_loss * calculate_area(10)

    vegetation_loss_area.plot.line('g-o', figsize=(11, 4))
    
    vegetation_loss_sum = vegetation_loss.sum('year')
    
    return vegetation_loss, vegetation_loss_sum    

In [ ]:
def plot_graph(vegetation_loss_sum, water_frequency_sum):
    
    backgroung_image = ds.isel(year =0).NDVI
    
    plt.figure(figsize=(12, 12))

    mining_area = vegetation_loss_sum.where(water_frequency_sum == True)
   
    mining_area = xr.where(mining_area >= 0, 1, 0)
    
    mining_area_vector = xr_vectorize(mining_area,
                              mask=mining_area.values == 1
                                 )
    
    mining_area_vector_buffer = mining_area_vector.buffer(250)
    
    mining_area_buffer = xr_rasterize(gdf=mining_area_vector_buffer,
                                  da=mining_area
                                     )
    
    vegetation_loss_new = vegetation_loss_sum.where(mining_area_buffer == True)
    
    vegetation_loss_new = xr.where(vegetation_loss_new > 0, True, False)
    
    vegetation_loss_new = vegetation_loss_new.where(vegetation_loss_new == True)
    
    water_observed_new = water_frequency_sum.where(mining_area_buffer == True)
    
    water_observed_new = xr.where(water_observed_new > 0, True, False)
    water_observed_new =water_observed_new.where(water_observed_new == True)
    
    backgroung_image.plot.imshow(cmap='Greys', add_colorbar=False)


    vegetation_loss_new.plot.imshow(cmap=ListedColormap(['Gold']), add_colorbar=False)
    water_observed_new.plot.imshow(cmap=ListedColormap(['Gold']), add_colorbar=False)

    plt.legend(
            [Patch(facecolor='Gold')], 
            ['Highly Potential Mining Site'],
            loc = 'upper left'
        )

    plt.title('Possible Mining Areas')

    plt.savefig('Mining_Chart')
    
    
    
    
    
    
    
    return vegetation_loss_new

In [ ]:
def plot_veg_mining(vegetation_loss, vegetation_loss_new):
    plt.figure(figsize=(12, 12))
    vegetation_loss_mininig = vegetation_loss.where((vegetation_loss == True) & (vegetation_loss_new == True))
    
    total_vegetation_loss_mininig = vegetation_loss_mininig.count(dim=['x', 'y'])
    total_vegetation_loss = vegetation_loss.where(vegetation_loss==True).count(dim=['x', 'y'])

    vegetation_loss_mininig_area = total_vegetation_loss_mininig * calculate_area(10)
    vegetation_loss_area = total_vegetation_loss * calculate_area(10)

    vegetation_loss_area.plot.line('g-o', figsize=(11, 4))
    vegetation_loss_mininig_area.plot.line('r-^')

    plt.legend(
            [Patch(facecolor='Green'), Patch(facecolor='Red')], 
            ['Vegetation Loss', 'Mining Contribution to Vegetation Loss'],
            loc = 'upper left'
        )

    plt.title('Plot Showing Mining Areas that contributed to Vegetation Loss')
    plt.ylabel('Area : km sq')
    plt.show()
    print('...................................................................')
    backgroung_image = ds.isel(year =0).NDVI
    plt.figure(figsize=(12, 12))
    
    size_n = vegetation_loss.year.size
    color_scheme = [name for name in mcd.TABLEAU_COLORS][:size_n]

    plt.figure(figsize=(12, 12))
    color_array = []
    year_array = []
    backgroung_image.plot.imshow(cmap='Greys', add_colorbar=False)

    for i in range(1, size_n):
        vegetation_loss_mininig.isel(year=i).plot.imshow(add_colorbar=False, cmap=ListedColormap([color_scheme[i]]))
        color_array.append(Patch(facecolor=f'{color_scheme[i]}'))
        year_array.append(f'{vegetation_loss.year.values[i]}')


    plt.legend(color_array, year_array, loc = 'upper left')

    plt.title(f'Mining Area that contribution to Vegetation Loss from {vegetation_loss.year.values[0]} to {vegetation_loss.year.values[-1]}')
    plt.show()